# Create CSV files with FHR from the binary files provided in the FHRMA toolbox

## About the CTG files
The FHRMA toolbox provides 66 training files and 90 test files with raw CTG data. They are stored as binary .fhr files. The signals were sampled at 4 Hz and stored in the following format. For each sample:
* A uint32 serves as the UNIX timestamp for the beginning of the recording
* A uint16 corresponds to the FHR from the first sensor (multiplied by 4)
* A uint16 for the FHR signal from the second sensor (multiplied by 4) (unused here)
* A uint8 for the TOCO signal (multiplied by 2)
* A uint8 reserved to store signal quality (unused here).

The FHR signal is set to 0 when it is missing.

## Background information on binary files
Binary files consist of sequential bytes, which are each 8 bits long. Bits are 0s and 1s.

Binary files typically have a header indicating the file type.

A byte can represent a number from 0 to 255, or -127 to +127 if you use one bit to indicate the minus sign. Groups of bytes can be used to represent larger numbers, for example:
| Number of bytes (or bits) | Name | Range (will - this to + this) |
| --- | --- | --- |
| 1 (8 bits) | Byte | 128 |
| 2 (16 bits) | Short | 32,768 |
| 4 (32 bits) | Int | 2,147,483,648 |
| 8 (64 bits) | Int64 | 9,223,372,036,854,775,808 |

## MATLAB function

The .fhr files can be opened with the fhropen.m function in MATLAB. I succesfully ran this in Octave, and have copied below:

```
clear;

% Set filename
filename='test02';

% Open file
f=fopen([filename, '.fhr'], 'r');

% Load timestamp for beginning of recording
timestamp=fread(f,1,'uint32');

% Load FHR data - first corresponds to first sensor, and
% second corresponds to second sensor (which they don't use)
% This reads 3 x 10,000,000 array
data=fread(f,[3,10000000],'uint16');
FHR1=data(1,:)/4;
FHR2=data(2,:)/4;

fseek(f,4,'bof');

% TOCO signal
data=fread(f,[6,10000000],'uint8');
TOCO=data(5,:)/2;

MHR=zeros(size(FHR1));
infos=[];
fclose(f);

% Save FHR1 to a file for use in python
csvwrite([filename, '.csv'], transpose(FHR1));
```

## Implementation in Python

Load required packages.

In [1]:
from dataclasses import dataclass
import numpy as np
import os
import glob

Define file paths.

In [2]:
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    fhrma = './fhrma/train_test_data/'
    train_fhr = 'traindata_fhr/'
    test_fhr = 'testdata_fhr/'
    train_csv = 'traindata_csv/'
    test_csv = 'testdata_csv/'


paths = Paths()

Define function for extracting FHR and saving to csv.

In [3]:
def convert_fhr_to_csv(fhr_path, csv_path):
    '''
    Imports .fhr files, extracts the data corresponding to FHR1, and saves as
    csv in the provided directory path
    Inputs:
    fhr_path - string, location of .fhr files
    csv_path - string, location to save the .csv files
    '''
    # Get list of all the .fhr file names in the directory
    files = glob.glob(os.path.join(fhr_path, '*.fhr'))

    # Loop through those files...
    for file_path in files:
        # Read file as uint16
        content = np.fromfile(file_path, dtype=np.uint16)

        # Ignore first two bytes, then reshape so you have a 3 x n array
        data = content[2:10000000-1].reshape(-1, 3).T

        # Take first array which corresponds to first sensor
        fhr1 = data[0]/4

        # Get file name (remove path, change to csv)
        name = file_path.replace(fhr_path, '').replace('.fhr', '.csv')

        # Save to csv file
        np.savetxt(os.path.join(csv_path, name), fhr1, delimiter=',')

Apply function to the training and test datasets.

In [4]:
convert_fhr_to_csv(
    fhr_path = os.path.join(paths.fhrma, paths.train_fhr),
    csv_path = os.path.join(paths.fhrma, paths.train_csv)
)

In [5]:
convert_fhr_to_csv(
    fhr_path = os.path.join(paths.fhrma, paths.test_fhr),
    csv_path = os.path.join(paths.fhrma, paths.test_csv)
)